# 使用 Nsight Systems 优化简单 DNN 训练程序

## 6.1 使用 Tensor Cores
作为本课程的一部分，您需要使用具有张量核心（Tensor Cores）的 Volta 或更新架构的 NVIDIA GPU。Tensor Cores 是专门为执行张量/矩阵运算而设计的专用执行单元，这些运算是深度学习中使用的核心计算功能。Tensor Cores 为矩阵计算提供了巨大的加速，并使 AI 程序员能够使用混合精度来实现更高的吞吐量，而不会牺牲准确性。

## 6.2 分析配置文件
为了验证应用程序是否使用张量核心，我们将使用 Nsight Systems 中的一项新功能：**GPU 性能指标采样**。请注意，在第 5.4 节中第二次优化后用于分析应用程序的 Nsight Systems 命令行中，我们使用了 `--gpu-metrics-device=all` CLI 选项。这启用了新功能的收集。它旨在衡量不同 GPU 子系统的利用率。GPU 内的硬件计数器会定期读取并用于生成性能指标。

数据以直观的可视化形式呈现在时间线上，可配置且易于浏览。数据以自上而下的层次结构方式组织，因此您可以在放大到单个问题区域之前观察整个分析会话。以前需要猜测和测试的问题现在可以一目了然地识别出来。

让我们通过检查 Nsight Systems GUI 中的报告 (`secondOptimization.nsys-rep`) 来分析应用程序的 Tensor Cores 使用情况。向下滚动到时间线的底部，直到看到 GPU 指标的时间线行，如下图所示。

<img src=images/TensorCoreUsage.jpg>

展开 `SM instructions` 时间线行以查看 `Tensor Active/ FP16 Active` ，它表示*SM 张量管道或 FP16x2 管道发出张量指令的周期数*与*采样周期内的周期数*的百分比。此处的百分比图为零，因此应用程序没有使用 GPU 上的 Tensor Cores，性能将受到影响。

## 6.3 优化代码以使用 Tensor Cores
PyTorch 有一个 [自动混合精度 (AMP)](https://pytorch.org/docs/stable/amp.html) 包，它为用户提供了一种将现有 FP32 训练脚本转换为混合 FP32/FP16 精度的简单方法。这可以使用 NVIDIA GPU 上的 Tensor Cores 实现更快的计算。执行下面的单元格以查看在 PyTorch 中使用 AMP 包所做的代码更改。

In [ ]:
!diff -U4 --color=always application/main_opt2.py application/main_opt3.py

## 6.4 再次进行分析以验证优化
让我们通过执行下面给出的单元再次进行分析，以验证代码更改是否启用了 Tensor Core 的使用。

In [ ]:
!nsys profile --trace cuda,osrt,nvtx \
--capture-range cudaProfilerApi \
--gpu-metrics-device=all \
--output /dli/task/nsys/thirdOptimization \
--force-overwrite true \
python3 /dli/task/nsys/application/main_opt3.py

在 GUI 中打开报告 (`thirdOptimization.nsys-rep`)。向下滚动以查看 `Tensor Active / FP16 Active` 时间线行。

<img src=images/Optimization3.jpg>

优化后的代码确实使用了 NVIDIA GPU 上的 Tensor Cores。

## 6.5 比较优化前后的性能
现在我们已经解决了三个不同的性能问题，让我们对应用程序 [main_opt3.py](application/main_opt3.py) 进行计时。

In [ ]:
!cd /dli/task/nsys/application && time python3 main_opt3.py

将运行 [步骤 02](02_pytorch_mnist.ipynb) 中的基准代码 [main_baseline_nvtx.py](application/main_baseline_nvtx.py) 所用的时间与应用了迄今为止的三个优化后的代码 [main_opt3.py](application/main_opt3.py) 所用的时间进行比较，我们发现总时间已从约 97 秒减少至约 63 秒。

我们可以继续优化过程，找出更多性能瓶颈并修改应用程序以解决这些问题，但为了节省时间，我们将在此停止。现在您可以转到此 Nsight Systems 课程的 [总结](07_summary.ipynb) 。